In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB

from scipy.interpolate import make_interp_spline

In [ ]:
def load_data():
    path = "./data/iris.data"
    df = pd.read_fwf(path, header = None)
    df = df[0].str.split(',',expand = True)
    df.iloc[ : , : 4] = df.iloc[ : , : 4].astype(float)
    return df

In [ ]:
def draw_Avg(df):
    avg = df.iloc[ : , : 4].mean()
    colors = ['red', 'green', 'blue', 'pink']
    feature = ["sepal length in cm", "sepal width in cm", "petal length in cm", "petal width in cm"]
    index = ['0', '1', '2', '3']
    
    x = range(4)
    for i in range(4):
        plt.bar(index[i], avg[i], color = colors[i], label = feature[i])
    
    plt.title("Average")
    plt.legend()
    plt.show()

In [ ]:
def draw_Std(df):
    Std = df.iloc[ : , : 4].std()
    colors = ['red', 'green', 'blue', 'pink']
    feature = ["sepal length in cm", "sepal width in cm", "petal length in cm", "petal width in cm"]
    index = ['0', '1', '2', '3']
    
    x = range(4)
    for i in range(4):
        plt.bar(index[i], Std[i], color = colors[i], label = feature[i])
    
    plt.title("Standard Deviation")
    plt.legend()
    plt.show()

In [ ]:
def draw_ValueCount(df):
    for i in range(4):
        feature = ["sepal length in cm", "sepal width in cm", "petal length in cm", "petal width in cm"]
        
        print("Feature \"{}\":".format(feature[i]))
        
        freq = df[i].value_counts().sort_index()
        freq.plot.hist()
        plt.title("Value Count (Total)")
        plt.show()
        
        df_I_s = df[(df[4] == "Iris-setosa")]
        freq_I_s = df_I_s[i].value_counts().sort_index()
        freq_I_s.plot.hist()
        plt.title("Value Count (Label is \"Iris-setosa\")")
        plt.show()
        
        print(":")
        df_I_ve = df[(df[4] == "Iris-versicolor")]
        freq_I_ve = df_I_ve[i].value_counts().sort_index()
        freq_I_ve.plot.hist()
        plt.title("Value Count (Label is \"Iris-versicolor\")")
        plt.show()
        
        df_I_vi = df[(df[4] == "Iris-virginica")]
        freq_I_vi = df_I_vi[i].value_counts().sort_index()
        freq_I_vi.plot.hist()
        plt.title("Value Count (Label is \"Iris-virginica\")")
        plt.show()

In [ ]:
df = load_data()
draw_Avg(df)
draw_Std(df)
draw_ValueCount(df)